# Segmenting and Clustering Neighborhoods in Toronto

## List of postal codes of Canada: M

All detail is available at below url

<a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"> List of Postal codes of Canada:</a>

Link is : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [71]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
print("Library imported")

Library imported


 Assiging url of wikipedia page to variable URL

In [72]:
URL="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

getting all content of wikipedia page

In [73]:
res=requests.get(URL).text

In [74]:
soup=BeautifulSoup(res,'lxml')

Creating a function to remove html element tag

In [75]:
import re
def remove_element(data):
    clean_tag=re.compile('<.*?>')
    clean_text=re.sub(clean_tag,'',data)
    return clean_text.replace('\n','')

### Retriving content of table

In [76]:
PostalCode=[]
Borough=[]
Neighborhood=[]
for item in soup.find('table',class_="wikitable").find_all('tr')[1::1]:
    data=item.find_all(['th','td'])
    try:
        PostalCode.append(remove_element(str(data[0])))
        Borough.append(remove_element(str(data[1])))
        Neighborhood.append(remove_element(str(data[2])))
    except IndexError:Pass

<b>counting the number of record found in each column</b>

In [77]:
print("PostalCode:",len(PostalCode),"Borough:",len(Borough),"Neighborhood",len(Neighborhood))

PostalCode: 180 Borough: 180 Neighborhood 180


In [78]:
postal_code_canada_column=["PostalCode","Borough","Neighborhood"]
dataset=pd.DataFrame(columns=postal_code_canada_column)

In [79]:
dataset["PostalCode"]=PostalCode
dataset["Borough"]=Borough
dataset["Neighborhood"]=Neighborhood

In [80]:
dataset.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [81]:
drop_index=dataset[dataset["Borough"]=="Not assigned"].index.to_list()

In [82]:
len(drop_index)

77

In [83]:
dataset.drop(index=drop_index,axis=0,inplace=True)

In [84]:
dataset.reset_index(inplace=True)

In [85]:
dataset.drop(columns=["index"],inplace=True)

In [86]:
dataset.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [87]:
geospatial_data=pd.read_csv("Geospatial_Coordinates.csv")

In [88]:
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [89]:
geo_column=geospatial_data.columns.to_list()

In [90]:
geo_column[0]="PostalCode"

In [91]:
geo_column

['PostalCode', 'Latitude', 'Longitude']

In [92]:
geospatial_data.columns=geo_column

In [93]:
geospatial_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [94]:
dataset_with_loc=pd.merge(dataset,geospatial_data,on="PostalCode")

In [95]:
dataset_with_loc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [96]:
import folium

In [109]:


longitude=-79.3883
latitude=43.6548

In [110]:
toranto_map=folium.Map(location=[latitude,longitude],zoom_start=11)

In [111]:
for lat, lng, borough, neighborhood in zip(dataset_with_loc['Latitude'], dataset_with_loc['Longitude'], dataset_with_loc['Borough'], dataset_with_loc['Neighborhood']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yello',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(toranto_map)  
    
toranto_map

In [112]:
toranto_data=dataset_with_loc[dataset_with_loc["Borough"]=="Downtown Toronto"].reset_index(drop=True)
toranto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [113]:
toranto_data.shape

(19, 5)

In [132]:
CLIENT_ID = 'KCAQMS2J5A5QPBQJVZDXJI33ZI0CL1PJ3D1PBQF2G5YRQDW4' # your Foursquare ID
CLIENT_SECRET = 'AD1UE4JHS4NJ20KRMET4RHZGLQKACRAJCRF4N3GVRCUNN50F' # your Foursquare Secret
VERSION = '20200704' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KCAQMS2J5A5QPBQJVZDXJI33ZI0CL1PJ3D1PBQF2G5YRQDW4
CLIENT_SECRET:AD1UE4JHS4NJ20KRMET4RHZGLQKACRAJCRF4N3GVRCUNN50F


In [127]:
toranto_data.loc[0,"Neighborhood"]
print(toranto_data.loc[0,"Longitude"])
neighborhood_longitude=toranto_data.loc[0,"Longitude"]
toranto_data.loc[0,"Longitude"]
neighborhood_latitude=toranto_data.loc[0,"Latitude"]
print(neighborhood_latitude,neighborhood_longitude)

-79.3606359
43.6542599 -79.3606359


In [133]:
LIMIT=100
radius=500
url="https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v{}&ll{},{}&radius={}&limit={}".format(
CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_latitude,
    neighborhood_longitude,radius,LIMIT
)

In [137]:
# type your answer here

LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [138]:
results=requests.get(url).json()

In [139]:
results

{'meta': {'code': 200, 'requestId': '5f005a340ab6c4151b69a3ff'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [140]:
venues=results['response']['groups'][0]['items']

In [143]:
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '54ea41ad498e9a11e9e13308',
   'name': 'Roselle Desserts',
   'location': {'address': '362 King St E',
    'crossStreet': 'Trinity St',
    'lat': 43.653446723052674,
    'lng': -79.3620167174383,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.653446723052674,
      'lng': -79.3620167174383}],
    'distance': 143,
    'postalCode': 'M5A 1K9',
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['362 King St E (Trinity St)',
     'Toronto ON M5A 1K9',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d16a941735',
     'name': 'Bakery',
     'pluralName': 'Bakeries',
     'shortName': 'Bakery',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'grou

In [144]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [145]:
import json

In [146]:
nearby_venues=json_normalize(venues)

C:\Users\anish\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [147]:
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']

In [148]:
nearby_venues=nearby_venues.loc[:,filtered_columns]

In [154]:
def get_category_type(row):
    try:
        category_list=row['categories']
    except:
        category_list=row['venue.categories']
    if len(category_list)==0:
        return None
    else:
        return category_list[0]['name']
    

In [155]:
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)

In [157]:
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
5,Corktown Common,Park,43.655618,-79.356211
6,Impact Kitchen,Restaurant,43.656369,-79.356980
7,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
8,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
9,The Distillery Historic District,Historic Site,43.650244,-79.359323


## Explore Neighborhoods in downtown Torento

In [158]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [159]:

torento_venues = getNearbyVenues(names=toranto_data['Neighborhood'],
                                   latitudes=toranto_data['Latitude'],
                                   longitudes=toranto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [160]:
torento_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
...,...,...,...,...,...,...,...
1212,Church and Wellesley,43.66586,-79.383160,Flash,43.664319,-79.380190,Strip Club
1213,Church and Wellesley,43.66586,-79.383160,Alexander Wood Statue,43.663702,-79.380301,Sculpture Garden
1214,Church and Wellesley,43.66586,-79.383160,Noah's Natural Foods,43.668532,-79.385885,Food & Drink Shop
1215,Church and Wellesley,43.66586,-79.383160,Croissant Tree,43.669575,-79.382331,Coffee Shop


In [161]:
torento_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,65,65,65,65,65,65
Christie,17,17,17,17,17,17
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [162]:
print('There are {} uniques categories.'.format(len(torento_venues['Venue Category'].unique())))

There are 206 uniques categories.


In [163]:
torento_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [164]:
torrento_onehot=pd.get_dummies(torento_venues[["Venue Category"]],prefix="",prefix_sep="")

In [165]:
torrento_onehot["Neighborhood"]=torento_venues['Neighborhood']

In [166]:
fixed_columns=[torrento_onehot.columns[-1]]+list(torrento_onehot.columns[:-1])

In [169]:
torrento_onehot=torrento_onehot[fixed_columns]

In [170]:
torrento_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [171]:
torrento_grouped=torrento_onehot.groupby('Neighborhood').mean().reset_index()
torrento_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.017241,0.000000,0.00,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Central Bay Street,0.015385,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.015385,0.000000,0.00,0.015385,0.000000,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,Church and Wellesley,0.026667,0.013333,0.000000,0.000000,0.000000,0.0,0.000000,0.013333,0.000000,...,0.013333,0.013333,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.013333,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.00,0.010000,0.000000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,...,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.00,0.010000,0.000000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.00,0.00,0.000000,0.010000,0.01,0.010000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.00,0.010000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.050000,0.000000,0.05,0.016667,0.000000,0.000000


In [172]:
num_top_venues = 5

for hood in torrento_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = torrento_grouped[torrento_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.03
3  Seafood Restaurant  0.03
4         Cheese Shop  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0  Airport Service  0.20
1   Airport Lounge  0.13
2    Boat or Ferry  0.07
3  Harbor / Marina  0.07
4         Boutique  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1       Sandwich Place  0.06
2   Italian Restaurant  0.06
3                 Café  0.05
4  Japanese Restaurant  0.05


----Christie----
                venue  freq
0       Grocery Store  0.24
1                Café  0.18
2                Park  0.12
3  Italian Restaurant  0.06
4               Diner  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.07
2  Japanese Restaurant  0.05
3       

In [173]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [174]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torrento_grouped['Neighborhood']

for ind in np.arange(torrento_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torrento_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Restaurant,Beer Bar,Seafood Restaurant,Bakery,Café,Beach,Shopping Mall
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Sculpture Garden,Boat or Ferry,Rental Car Location,Boutique,Bar,Harbor / Marina,Coffee Shop,Airport Terminal
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Japanese Restaurant,Café,Thai Restaurant,Bar,Bubble Tea Shop,Burger Joint,Department Store
3,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Diner,Italian Restaurant,Candy Store,Athletics & Sports,Restaurant
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Mediterranean Restaurant,Men's Store,Hotel,Bubble Tea Shop


In [176]:
from sklearn.cluster import KMeans

In [177]:
# set number of clusters
kclusters = 5

torrento_grouped_clustering = torrento_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torrento_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 2, 4, 2, 2, 2, 2, 2, 2])

In [178]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torrento_merged = toranto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torrento_merged = torrento_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

torrento_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Dessert Shop,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Japanese Restaurant,Ramen Restaurant,Bookstore,Bakery
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Gastropub,Restaurant,American Restaurant,Cocktail Bar,Gym,Clothing Store,Hotel,Italian Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Cheese Shop,Restaurant,Beer Bar,Seafood Restaurant,Bakery,Café,Beach,Shopping Mall


In [180]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [181]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torrento_merged['Latitude'], torrento_merged['Longitude'], torrento_merged['Neighborhood'], torrento_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [182]:
torrento_merged.loc[torrento_merged['Cluster Labels'] == 0, torrento_merged.columns[[1] + list(range(5, torrento_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Dessert Shop,Restaurant
1,Downtown Toronto,0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center


In [183]:
torrento_merged.loc[torrento_merged['Cluster Labels'] == 1, torrento_merged.columns[[1] + list(range(5, torrento_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,Park,Playground,Trail,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [184]:
torrento_merged.loc[torrento_merged['Cluster Labels'] == 2, torrento_merged.columns[[1] + list(range(5, torrento_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Japanese Restaurant,Ramen Restaurant,Bookstore,Bakery
3,Downtown Toronto,2,Café,Coffee Shop,Gastropub,Restaurant,American Restaurant,Cocktail Bar,Gym,Clothing Store,Hotel,Italian Restaurant
4,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Cheese Shop,Restaurant,Beer Bar,Seafood Restaurant,Bakery,Café,Beach,Shopping Mall
5,Downtown Toronto,2,Coffee Shop,Sandwich Place,Italian Restaurant,Japanese Restaurant,Café,Thai Restaurant,Bar,Bubble Tea Shop,Burger Joint,Department Store
7,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Deli / Bodega,Clothing Store,Salad Place,Bakery
8,Downtown Toronto,2,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Italian Restaurant,Sporting Goods Shop,Scenic Lookout,Fried Chicken Joint,Brewery
9,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Salad Place,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega
10,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Beer Bar
11,Downtown Toronto,2,Café,Japanese Restaurant,Restaurant,Bar,Italian Restaurant,Bookstore,Bakery,Beer Bar,Beer Store,Sandwich Place
12,Downtown Toronto,2,Café,Bakery,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Pizza Place,Gaming Cafe,Bar


In [185]:
torrento_merged.loc[torrento_merged['Cluster Labels'] == 3, torrento_merged.columns[[1] + list(range(5, torrento_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,3,Airport Service,Airport Lounge,Sculpture Garden,Boat or Ferry,Rental Car Location,Boutique,Bar,Harbor / Marina,Coffee Shop,Airport Terminal


In [186]:
torrento_merged.loc[torrento_merged['Cluster Labels'] == 4, torrento_merged.columns[[1] + list(range(5, torrento_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,4,Grocery Store,Café,Park,Coffee Shop,Nightclub,Diner,Italian Restaurant,Candy Store,Athletics & Sports,Restaurant
